<a href="https://colab.research.google.com/github/mihayy/review_aspect_score_prediction/blob/master/models/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Models

### import libs

In [2]:
!pip install opt-einsum==2.3.2 tb-nightly==1.15.0a20190730 tf-estimator-nightly==1.14.0.dev2019073001 tf-nightly-gpu==1.15.0.dev20190730

     |████████████████████████████████| 61kB 25.8MB/s 
     |████████████████████████████████| 4.1MB 43.9MB/s 
     |████████████████████████████████| 501kB 47.6MB/s 
     |████████████████████████████████| 406.7MB 62kB/s 
  Created wheel for opt-einsum: filename=opt_einsum-2.3.2-cp36-none-any.whl size=49883 sha256=a85415f72ec0397f28c6bc7e60fb64e59d1e7a63410c8dab87b7d65f3afd9d7f
  Stored in directory: /root/.cache/pip/wheels/51/3e/a3/b351fae0cbf15373c2136a54a70f43fea5fe91d8168a5faaa4
Successfully built opt-einsum
  Found existing installation: opt-einsum 3.0.1
    Uninstalling opt-einsum-3.0.1:
      Successfully uninstalled opt-einsum-3.0.1


In [3]:
import nltk
import torchtext
from nltk.tokenize import sent_tokenize, word_tokenize
from tensorflow.contrib.layers import fully_connected
import tensorflow as tf
from google.colab import files
import os
from functools import partial
import numpy as np
from pathlib import Path

from tensorflow.python.ops.metrics_impl import _streaming_confusion_matrix

from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
nltk.download('stopwords')

%load_ext tensorboard
# %load_ext tensorboard.notebook

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
!pip list

Package                  Version              
------------------------ ---------------------
absl-py                  0.7.1                
alabaster                0.7.12               
albumentations           0.1.12               
altair                   3.2.0                
astor                    0.8.0                
astropy                  3.0.5                
atari-py                 0.1.15               
atomicwrites             1.3.0                
attrs                    19.1.0               
audioread                2.1.8                
autograd                 1.3                  
Babel                    2.7.0                
backcall                 0.1.0                
backports.tempfile       1.0                  
backports.weakref        1.0.post1            
beautifulsoup4           4.6.3                
bleach                   3.1.0                
blis                     0.2.4                
bokeh                    1.0.4                
boto         

In [0]:
device_name = tf.test.gpu_device_name()
if device_name != "/device:GPU:0":
  print("Device not found!")
else:
  print(device_name)

/device:GPU:0


### Mount Google Drive

In [4]:
from google.colab import drive
drive.mount('gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at gdrive


### General environment variables

In [0]:
class Config(object):
  def __init__(self):
    self.gdrive_working_dir = Path("gdrive/My Drive/thesis")

    self.embeddings_path = self.gdrive_working_dir/ "embeddings"

#     self.dataset_intermediate_path = Path("manual_labeled_strength_weak_sections/summary_strength_weak_sections")
    self.dataset_intermediate_path = Path("")

#     self.dataset_intermediate_path = Path("manual_labeled_strength_weak_sections")
#     self.dataset_intermediate_path = Path("acl_abstracts")
    self.dataset_path = self.gdrive_working_dir/ "data" / self.dataset_intermediate_path

    self.train_dataset_filename = "train_dataset.csv"
    self.dev_dataset_filename = "dev_dataset.csv"
    self.test_dataset_filename = "acl_dev_test.csv"
    
    self.train_ds = "train_ds_"
    self.test_ds = "test_ds_"

    self.aspects_no_com_approp = ['RECOMMENDATION', 'REVIEWER_CONFIDENCE', 'SOUNDNESS_CORRECTNESS', 'IMPACT', 'SUBSTANCE', 'CLARITY', 'ORIGINALITY']
    
    self.stats_path = self.gdrive_working_dir / "tensorflow_stats_random"
#     self.stats_path = Path("")
    
  def set_dataset_intermediate_path(self, intermediate_path):
    self.dataset_intermediate_path = intermediate_path
    self.dataset_path = self.gdrive_working_dir/ "data" / self.dataset_intermediate_path
    
  def set_ds_fname(self, iteration):
    self.train_dataset_filename = self.train_ds + str(iteration) + ".csv"
    self.dev_dataset_filename = self.test_ds + str(iteration) + ".csv"

In [0]:
env_var = Config()

### Dataset manipulation

#### Section manipulation

In [0]:
import pandas as pd
from string import punctuation
import re

In [0]:
class Review_Section(object):
  def __init__(self):
    self.any_strength_weak_sections = False
    self.is_strength_present = False
    self.is_weakness_present = False
    self.is_discussion_present = False

    self.strength_idx = 0
    self.weakness_idx = 0
    self.discussion_idx = None

    self.relevant_sections = []
    self.init_sections = []
    self.first_section = ""
    self.last_section = ""
    self.filtered_in_between_sections = []
    
    self.irrelevant_section_keywords = ["minor", "detailed", "reference", "other", "question", "small", "author", "further", "additional"]
    
    self.section_len = 170
    
  def try_detect_strength_weak_sections(self, sections):
    for idx, section in enumerate(sections):
      strip_lower_section = section.strip().lower()
      if "weak" in strip_lower_section:
        self.is_weakness_present = True
        self.any_strength_weak_sections = True
        self.weakness_idx = idx

      if "stren" in strip_lower_section:
        self.is_strength_present = True
        self.any_strength_weak_sections = True
        self.strength_idx = idx

      if "general discussion" in strip_lower_section:
        self.is_discussion_present = True     
        self.discussion_idx = idx
  
  def compute_strength_section(self, sections):
    if self.is_strength_present:
      self.first_section = sections[self.strength_idx]

    in_between_sections = sections[self.strength_idx:self.discussion_idx] 

    if self.is_discussion_present:
      self.last_section = sections[self.discussion_idx]

    self.relevant_sections = self.init_sections

    for section in in_between_sections:
      section_strip_lower = section.strip().lower()
      is_section_irrelevant = [keyword in section_strip_lower for keyword in self.irrelevant_section_keywords + ["weakness"]]
      if any(is_section_irrelevant):
        self.last_section = section
        break  
      self.relevant_sections.append(section)
      
  def compute_summary_section(self, sections):
    in_between_sections = sections[:self.discussion_idx] 
    
    if self.is_discussion_present:
      self.last_section = sections[self.discussion_idx]
    
    if self.is_strength_present:
      self.last_section = sections[self.strength_idx]
      in_between_sections = sections[:self.strength_idx] 

    self.relevant_sections = self.init_sections

    for section in in_between_sections:
      section_strip_lower = section.strip().lower()
      is_section_irrelevant = [keyword in section_strip_lower for keyword in self.irrelevant_section_keywords + ["weakness"]]
      if any(is_section_irrelevant):
        self.last_section = section
        break  
      self.relevant_sections.append(section)
      
  def compute_weakness_section(self, sections):

    in_between_sections = sections[:self.discussion_idx]

    if self.is_discussion_present:
      self.last_section = sections[self.discussion_idx]

    if self.is_weakness_present:
      self.first_section = sections[self.weakness_idx]
      
      in_between_sections = sections[self.weakness_idx+1:self.discussion_idx] 
      
    self.relevant_sections = self.init_sections

    for section in in_between_sections:
      section_strip_lower = section.strip().lower()
      is_section_irrelevant = [keyword in section_strip_lower for keyword in self.irrelevant_section_keywords]
      if any(is_section_irrelevant):
        self.last_section = section
        break  
      self.relevant_sections.append(section)
  
  def compute_strength_weak_section(self, sections, should_exclude_first_section=True):
    if self.is_strength_present and should_exclude_first_section:
      self.first_section = sections[self.strength_idx]

    in_between_sections = sections[self.strength_idx:self.discussion_idx] 

    if self.is_discussion_present:
      self.last_section = sections[self.discussion_idx]

    if self.is_weakness_present:
      self.init_sections = sections[self.strength_idx:self.weakness_idx+1]
      in_between_sections = sections[self.weakness_idx+1:self.discussion_idx] 
      
    self.relevant_sections = self.init_sections

    for section in in_between_sections:
      section_strip_lower = section.strip().lower()
      is_section_irrelevant = [keyword in section_strip_lower for keyword in self.irrelevant_section_keywords]
      if any(is_section_irrelevant):
        self.last_section = section
        break  
      self.relevant_sections.append(section)
  
  def merge_relevant_sections(self, sections, review_text, should_limit_strength_weak_section_len=False, should_move_weak_section_first=False):
    if should_limit_strength_weak_section_len and self.is_weakness_present:
      return self.limit_strength_weak_section_len(sections, review_text, should_move_weak_section_first)
    else:
      start_idx = review_text.find(self.first_section) if self.first_section else 0
      end_idx = review_text.find(self.last_section)  if self.last_section else None

      return review_text[start_idx:end_idx]
      
  def limit_strength_weak_section_len(self, sections, review_text, should_move_weak_section_first):
      
    strength_start_idx = review_text.find(self.first_section) if self.first_section else 0
    strength_end_idx = review_text.find(sections[self.weakness_idx])

    weak_end_idx = review_text.find(self.last_section) if self.last_section else None

    strength_section_text = review_text[strength_start_idx:strength_end_idx]
    weak_section_text = review_text[strength_end_idx:weak_end_idx]

    strength_section_split = strength_section_text.split(" ")
    if len(strength_section_split) > self.section_len:
      strength_section_split = strength_section_split[:self.section_len]
      strength_section_text = " ".join(strength_section_split)

    weak_section_split = weak_section_text.split(" ")
    if len(weak_section_split) > self.section_len:
      weak_section_split = weak_section_split[:self.section_len]
      weak_section_text = " ".join(weak_section_split)

    if should_move_weak_section_first:
      return weak_section_text + strength_section_text
      
    return strength_section_text + weak_section_text
      
def review_pruning(text_column):
  section_detector_tokenizer = RegexpTokenizer('\n\n.{6,45}?:')
  section_detector_relaxed_tokenizer = RegexpTokenizer('\n\n.{6,45}?\n\n?')

  patterned_review_count = 0
  unpatterned_review_count = 0
  
  patterned_review_idx = []

  review_text_relevant_sections = []

  for i in range(len(text_column)):
    
    review_text = text_column[i]
    review_section = Review_Section()
    
    sections = section_detector_tokenizer.tokenize(review_text)
    if len(sections):
      review_section.try_detect_strength_weak_sections(sections)

    if not review_section.any_strength_weak_sections:
      sections = section_detector_relaxed_tokenizer.tokenize(review_text)
      review_section.try_detect_strength_weak_sections(sections)

    if review_section.any_strength_weak_sections:
      patterned_review_idx.append(i)
      patterned_review_count += 1
    else:
      unpatterned_review_count += 1

    if len(sections):
      review_section.compute_strength_weak_section(sections)
#       review_section.compute_summary_section(sections)
#       review_section.compute_weakness_section(sections)
#       review_section.compute_strength_section(sections)
#       review_section.compute_strength_weak_section(sections, should_exclude_first_section=False)
      
#     review_text = review_section.merge_relevant_sections(sections, review_text)
#     review_text = review_section.merge_relevant_sections(sections, review_text, should_limit_strength_weak_section_len=True)
    review_text = review_section.merge_relevant_sections(sections, review_text, should_limit_strength_weak_section_len=True,  should_move_weak_section_first=True)
    
    review_text_relevant_sections.append(review_text)

  print(patterned_review_count)
  print(unpatterned_review_count)
  print(len(text_column))
  
  return review_text_relevant_sections, patterned_review_idx

In [0]:
current_file = env_var.test_dataset_filename
ds_df = pd.read_csv(env_var.dataset_path / current_file)
ds_relevant_sections, patterned_review_idx = review_pruning(ds_df["comments"])
ds_df["comments"] = ds_relevant_sections
## ds_df = ds_df.iloc[patterned_review_idx]
# ds_df.head()
# ds_df.to_csv(env_var.dataset_path / "weak_strength_sections_len_limit" / current_file, index=False) 

18
9
27


In [0]:
# copy strength_weak_sections_len_limit ds splits

for i in range(10):
  filename = "train_ds_" + str(i) + ".csv" 
  
  path_with_filename = env_var.dataset_path / "manual_labeled_strength_weak_sections" / "aug_stren_weak_sections_len_limit" / filename

  strength_weak_sections_df = pd.read_csv(path_with_filename)

  to_aug_ds_part = pd.read_csv(path_with_filename)
  ds_relevant_sections, patterned_review_idx = review_pruning(to_aug_ds_part["comments"])
  to_aug_ds_part["comments"] = ds_relevant_sections

  strength_weak_sections_aug_df = pd.concat([strength_weak_sections_df, to_aug_ds_part])
  aug_train_ds = strength_weak_sections_aug_df.drop_duplicates()
#   aug_train_ds.to_csv(path_with_filename, index=False) 
  print(len(aug_train_ds))
  print()

#### Clarity sentence manipulation

In [0]:
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import pandas as pd

def get_clarity_review_text_column(text_column, clarity_presence_regex_detection_list):
  clarity_coverage = 0
  clarity_review_text_column = []
  indices = []
  for i in range(len(text_column)):
    review_text = text_column[i]
    review_text_sentences = sent_tokenize(review_text)
    clarity_sentences = []
    is_clarity_detected_in_review = False
    
    for sentence in review_text_sentences:
      normalize_sentence = sentence.lower().strip().replace("\n", " ")
      is_clarity_detected = any(re.match(regex, normalize_sentence) for regex in clarity_presence_regex_detection_list)
      if is_clarity_detected:
        clarity_sentences.append(sentence)
        is_clarity_detected_in_review = True
        
    if is_clarity_detected_in_review:
      clarity_review_text_column.append(" ".join(clarity_sentences))
      clarity_coverage += 1
      indices.append(i)
    else:
      clarity_review_text_column.append(review_text)
      
  print(clarity_coverage, "/", len(text_column), "/", clarity_coverage / len(text_column))
  return indices, clarity_review_text_column
  
paper_is_adjective = re.compile(".*?paper +is +(?:\w+ +)?(clear|unclear|thoroughly|extremely|easy|well|nice|nicely|clearly|poorly|\w+ly).*?")
positive_written = re.compile(".*?(nicely|well).*?written.*?")
clarity_keywords = re.compile(".*?(typo.? +|understandable|thoroughly|structured|thorough|explanation|organized|misleading|mistakes|orthographical|grammar|spelling|readability|cohesion|explained|reads|written|clarity|clear|clearly).*?")
adjective_described = re.compile(".*?(poorly|well|thoroughly|nicely|clearly|\w+ly) +described.*?")
adjective_to_read = re.compile(".*?(hard|easy) +to +read.*?")
language_error = re.compile(".*?language +error.*?")
exists_paper_read = re.compile(".*?(?=.*?paper)(?=.*?read).*")
native_speaker = re.compile(".*?native +speaker.*?")
adjective_to_understand = re.compile(".*?(hard|fail|difficult) +to +(understand|interpret).*?")

clarity_presence_regex_detection_list = [paper_is_adjective, positive_written, clarity_keywords, adjective_described, adjective_to_read, language_error, exists_paper_read, native_speaker, adjective_to_understand]

current_file = env_var.test_dataset_filename
aug_train_ds = pd.read_csv(env_var.dataset_path / current_file)
indices, clarity_review_text_column = get_clarity_review_text_column(aug_train_ds["comments"], clarity_presence_regex_detection_list)
aug_train_ds["comments"] = clarity_review_text_column
# aug_train_ds
# aug_train_ds.iloc[indices].to_csv(env_var.dataset_path / "clarity_sentences/mandatory_clarity_sentences" / current_file, index=False) 
# aug_train_ds.to_csv(env_var.dataset_path / "clarity_sentences" / current_file, index=False) 

210 / 269 / 0.7806691449814126


### Data preprocesssing, torchtext 

#### TorchText

In [0]:
from torchtext import data

lower=True
only_char=True
stop_remove=False

def tokenizer(review_text):
    if lower: 
        review_text = review_text.lower()
    if only_char:
        regex_tokenizer = nltk.RegexpTokenizer(r'\w+')
        tokens = regex_tokenizer.tokenize(review_text)
        review_text = ' '.join(tokens)
    tokens = word_tokenize(review_text)
    if stop_remove:
        tokens = [w for w in tokens if not w in nltk.corpus.stopwords.words('english')]
  
    return tokens

def get_tabular_datasets(seq_length):
    comments_field = data.Field(sequential=True,
                                tokenize=tokenizer,
                                include_lengths=True,
                                use_vocab=True,
                                fix_length=seq_length)

    grade_field = data.Field(sequential=False, 
                             use_vocab=False, 
                             pad_token=None,
                             unk_token=None)
    
    dataset_fields = [
        ('comments', comments_field), # process it as text
        ('RECOMMENDATION', grade_field), # process it as label
        ('REVIEWER_CONFIDENCE', grade_field),
        ('SOUNDNESS_CORRECTNESS', grade_field),
        ('IMPACT', grade_field),
        ('SUBSTANCE', grade_field),
        ('CLARITY', grade_field),
        ('ORIGINALITY', grade_field)
    ]

    tabular_train, tabular_dev, tabular_test = data.TabularDataset.splits(path=env_var.dataset_path,
                                                                          format='csv', 
                                                                          train=env_var.train_dataset_filename, 
                                                                          validation=env_var.dev_dataset_filename, 
                                                                          test=env_var.test_dataset_filename,
                                                                          fields=dataset_fields, 
                                                                          skip_header=True)
    return tabular_train, tabular_dev, tabular_test, comments_field

          
def get_vocab(comments_field, 
              tabular_train,
              vocab_max_size,
              vocab_min_freq,
              pretrained_vectors,
              mean=0, 
              std=0.01):
    
    unk_init_normal_ = lambda input_tensor: input_tensor.normal_(mean=mean, std=std)
    
    comments_field.build_vocab(
        tabular_train,
        max_size=vocab_max_size,
        min_freq=vocab_min_freq,
        vectors=pretrained_vectors,
        unk_init=unk_init_normal_)
    
    return comments_field.vocab

def get_batches_iterators(tabular_train, tabular_dev, tabular_test, 
                          comments_field_name,
                          train_batch_size,
                          dev_batch_size,
                          test_batch_size):
    # CREATE BATCH ITERATORS

    train_batch, dev_batch = data.BucketIterator.splits(datasets=(tabular_train, tabular_dev), # specify train and validation Tabulardataset
                                                        batch_sizes=(train_batch_size, dev_batch_size),  # batch size of train and validation
                                                        sort_key=lambda x: len(x.comments), # on what attribute the text should be sorted
                                                        sort_within_batch=True,
                                                        repeat=False)

    test_batch = data.Iterator(tabular_test, batch_size=test_batch_size, sort=False, sort_within_batch=False, repeat=False)

    # BATCH DECORATOR 

    class BatchWrapper:
        def __init__(self, dataset_batch, x_field):
            self.dataset_batch, self.x_field = dataset_batch, x_field

        def __len__(self):
            return len(self.dataset_batch)

        def __iter__(self):
            for batch in self.dataset_batch:
                X, batch_seq_len = getattr(batch, self.x_field)
    #             because tensorflow expects this tranposing
                X = X.transpose(0,1).numpy()
                batch_seq_len = batch_seq_len.numpy()
                y = {}
                for aspect in env_var.aspects_no_com_approp:
                  y[aspect] = getattr(batch, aspect).numpy()
                yield (X,y,batch_seq_len)

    train_batch_it = BatchWrapper(train_batch, comments_field_name) 
    dev_batch_it = BatchWrapper(dev_batch, comments_field_name) 
    test_batch_it = BatchWrapper(test_batch, comments_field_name) 
    
    return train_batch_it, dev_batch_it, test_batch_it

In [0]:
def preprocessing_pipeline(seq_length, vocab_max_size, vocab_min_freq, pretrained_vectors, comments_field_name):
  tabular_train, tabular_dev, tabular_test, comments_field = get_tabular_datasets(seq_length=seq_length)
  vocab = get_vocab(comments_field, tabular_train, vocab_max_size=vocab_max_size, vocab_min_freq=vocab_min_freq, pretrained_vectors=pretrained_vectors)
  
  train_batch_size = len(tabular_train)
  dev_batch_size = len(tabular_dev)
  test_batch_size = len(tabular_test)

  train_batch_it, dev_batch_it, test_batch_it = get_batches_iterators(tabular_train, tabular_dev, tabular_test, 
                                                                      comments_field_name=comments_field_name,
                                                                      train_batch_size=train_batch_size,
                                                                      dev_batch_size=dev_batch_size,
                                                                      test_batch_size=test_batch_size)
  embedding_dim = vocab.vectors.shape[1]
  vocab_size = len(vocab)
  return vocab, vocab_size, embedding_dim, train_batch_it, dev_batch_it, test_batch_it

## Models

### Helper base class

In [0]:
def get_last_output(outputs, seq_length=None):
    nr_batch_instances = tf.shape(outputs)[0]
    batch_range = tf.range(nr_batch_instances)
    last_idx_array = - tf.ones([nr_batch_instances],tf.int32) + tf.shape(outputs)[1]
    
    if seq_length is not None:
        last_idx_array = seq_length - 1
    
    batch_range = tf.stack([batch_range,last_idx_array], axis=1)
    
    return tf.gather_nd(outputs, batch_range)

In [0]:
class RNN_base_model(object):
  def __init__(self):
    self.batch_seq_len = tf.placeholder(tf.int32, [None], name="seq_len")
    self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    self.keep_prob_input = tf.placeholder(tf.float32, name='keep_prob_input')
    self.keep_prob_cell = tf.placeholder(tf.float32, name='keep_prob_cell')
    self.is_training = tf.placeholder(tf.bool, name="is_training");
    
    self.dropout_layer = partial(tf.contrib.layers.dropout, keep_prob=self.keep_prob)    
    self.input_dropout_layer = partial(tf.contrib.layers.dropout, keep_prob=self.keep_prob_input)    
    self.batch_norm_layer = partial(tf.layers.batch_normalization, training=self.is_training, momentum=0.9)
    self.fully_connected = partial(tf.contrib.layers.fully_connected, activation_fn=None)
    
    self.metrics_scope_name = "metrics"
    
  def embedding_layer_init(self, vocab_size, embedding_dim): 
    
    self.embedding_placeholder = tf.placeholder(tf.float32, [vocab_size, embedding_dim], name="embedding_placeholder")

    embedding = tf.get_variable('embedding', [vocab_size, embedding_dim], dtype=tf.float32, trainable=True)

    self.embedding_init = embedding.assign(self.embedding_placeholder)

    return embedding
    
  def rnn_encoder_layer(self, embedding_inputs, n_neurons, n_layers, cell_type, batch_seq_len, use_bidirectional):    
    def cell_with_droput(layer_nr, name_suffix=""):
      cell_name = name_suffix + "cell_" + cell_type + "_" + str(layer_nr) 
      cell = self.get_cell(n_neurons, cell_type, cell_name)
      
      dropout_keep_prob = 1.0 if layer_nr == 0 else self.keep_prob_cell
      
      return tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=dropout_keep_prob)

    if use_bidirectional:
      with tf.name_scope("bi_rnn_layer"):

        cell_fw = tf.contrib.rnn.MultiRNNCell([cell_with_droput(i, "fw") for i in range(n_layers)])    
        cell_bw = tf.contrib.rnn.MultiRNNCell([cell_with_droput(i, "bw") for i in range(n_layers)])    

        (fw_pass, bw_pass), states = tf.nn.bidirectional_dynamic_rnn(cell_fw=cell_fw, cell_bw=cell_bw, inputs=embedding_inputs, dtype=tf.float32, sequence_length=batch_seq_len)

        outputs = tf.concat([get_last_output(fw_pass, batch_seq_len), get_last_output(bw_pass, batch_seq_len)], 1)
    else:
      with tf.name_scope("rnn_layer"):
        cell = tf.contrib.rnn.MultiRNNCell([cell_with_droput(i) for i in range(n_layers)])    

        outputs, states = tf.nn.dynamic_rnn(cell, embedding_inputs, dtype=tf.float32, sequence_length=batch_seq_len)
        outputs = get_last_output(outputs, batch_seq_len)
      
    return outputs
  
  def optimize_loss(self, loss, learning_rate, weight_decay):
#     optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    optimizer = tf.contrib.opt.AdamWOptimizer(weight_decay=weight_decay, learning_rate=learning_rate)
    self.training_op = optimizer.minimize(loss)
  
  def compute_list_mean(self, input_list):
    return tf.reduce_mean(tf.convert_to_tensor(input_list, dtype=tf.float32))
  
  def get_loss(self, labels, logits):
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))
  
  def get_accuracy(self, labels, logits):
#     self.rmsqe = tf.sqrt(tf.losses.mean_squared_error(tf.cast(self.y_impact, tf.int64), tf.argmax(input=logits, axis=1)))
    correct = tf.nn.in_top_k(logits, labels, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    return accuracy
  
  def get_epoch_accuracy(self, labels, logits):
    return tf.metrics.accuracy(labels=tf.cast(labels, tf.int64), predictions=tf.argmax(input=logits, axis=1))
  
  def get_epoch_conf_matrix(self, labels, logits):
    return _streaming_confusion_matrix(tf.cast(labels, tf.int32), tf.argmax(input=logits, axis=1), 5)
  
  def register_metrics_init_op(self):
    metric_variables = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope=self.metrics_scope_name)
    self.metrics_init_op = tf.variables_initializer(metric_variables)
  
  def get_cell(self, n_neurons, cell_type, cell_name):
    if cell_type == "rnn":
      return tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, name=cell_name)
    
    if cell_type == "gru":
      return tf.contrib.rnn.GRUCell(num_units=n_neurons, name=cell_name)

    if cell_type == "lstm":
      return tf.contrib.rnn.LSTMCell(num_units=n_neurons, name=cell_name)

    if cell_type == "lstm_batch_norm":
      return tf.contrib.rnn.LayerNormBasicLSTMCell(num_units=n_neurons)
    
    if cell_type == "attention":
      cell = tf.contrib.rnn.GRUCell(num_units=n_neurons, name=cell_name)
      return tf.contrib.rnn.AttentionCellWrapper(cell, 3)
    
    return tf.contrib.rnn.LSTMCell(num_units=n_neurons, use_peepholes=True, name=cell_name)

### RNN single aspect learning model

In [0]:
class RNN_single_aspect_model(RNN_base_model):
  def __init__(self, model_config, vocab_size, embedding_dim, n_steps):
    with tf.device("/gpu:0"):
        super().__init__()
        
        self.X = tf.placeholder(tf.int32, [None, n_steps], name="X")
        self.y = tf.placeholder(tf.int32, [None], name="y")
        
        embedding = self.embedding_layer_init(vocab_size, embedding_dim)       
    
        X_embeddings = tf.nn.embedding_lookup(embedding, self.X)
      
        X_embeddings = self.input_dropout_layer(X_embeddings)

        outputs = self.rnn_encoder_layer(X_embeddings, model_config.n_neurons, model_config.n_layers, model_config.cell_type, self.batch_seq_len, model_config.use_bidirectional)
        
        outputs = self.batch_norm_layer(outputs)
        
        if model_config.use_extra_dense_layer:
          outputs = self.dropout_layer(outputs)
          outputs = self.fully_connected(outputs, model_config.n_neurons)
          outputs = self.batch_norm_layer(outputs)
          outputs = tf.nn.relu(outputs)
      
        outputs = self.dropout_layer(outputs)
        logits = self.fully_connected(outputs, model_config.n_outputs)

        self.batch_loss = self.get_loss(self.y, logits)
    
        self.optimize_loss(self.batch_loss, model_config.learning_rate, model_config.weight_decay)
        
        self.batch_accuracy = self.get_accuracy(self.y, logits)
        
        with tf.variable_scope(self.metrics_scope_name):
          self.accuracy, accuracy_update_op = self.get_epoch_accuracy(self.y, logits)
          self.loss, loss_update_op = tf.metrics.mean(self.batch_loss)
          self.conf_matrix, conf_matrix_update_op = self.get_epoch_conf_matrix(self.y, logits)
          
        self.update_metrics_op = tf.group([accuracy_update_op, loss_update_op, conf_matrix_update_op])
        
        self.register_metrics_init_op()
        
        tf.summary.scalar("accuracy", self.accuracy)
        tf.summary.scalar("loss", self.loss)
        
        self.summary_op = tf.summary.merge_all()

        self.init = tf.global_variables_initializer()

  def feed_dict_constructor(self, batch_comments, batch_label, batch_seq_len, droput_keep_prob, droput_keep_prob_input, droput_keep_prob_cell, is_training, aspect):
    return {self.X: batch_comments, 
            self.y: batch_label[aspect] - 1,
            self.batch_seq_len: batch_seq_len,
            self.keep_prob:droput_keep_prob,
            self.keep_prob_input: droput_keep_prob_input,
            self.keep_prob_cell: droput_keep_prob_cell,
            self.is_training: is_training}
  
  def perform_train_op(self, sess, train_batch_item, droput_keep_prob, droput_keep_prob_input, droput_keep_prob_cell, aspect):
    sess.run(self.training_op, feed_dict=self.feed_dict_constructor(*train_batch_item, droput_keep_prob, droput_keep_prob_input, droput_keep_prob_cell, True, aspect))
  
  def update_batch_metrics(self, sess, batch_item, droput_keep_prob, aspect):
    sess.run(self.update_metrics_op, feed_dict=self.feed_dict_constructor(*batch_item, droput_keep_prob, droput_keep_prob, droput_keep_prob, False, aspect))
    
  def get_epoch_metrics(self, sess, aspect):
    metric_op = [self.conf_matrix, self.accuracy, self.loss, self.summary_op]
    conf_matrix, accuracy, loss, summary = sess.run(metric_op)
    return conf_matrix, accuracy, loss, summary

### RNN multi task aspect learning model

In [0]:
class RNN_multi_task_model(RNN_base_model):
  def __init__(self, model_config, vocab_size, embedding_dim, n_steps):
    with tf.device("/gpu:0"):
        super().__init__()
        
        n_neurons = model_config.n_neurons
        n_outputs = model_config.n_outputs
        
        embedding = self.embedding_layer_init(vocab_size, embedding_dim)
        
        self.X = tf.placeholder(tf.int32, [None, n_steps], name="X")
    
        X_embeddings = tf.nn.embedding_lookup(embedding, self.X)
        
        X_embeddings = self.input_dropout_layer(X_embeddings)
        
        self.y_rec = tf.placeholder(tf.int32, [None], name="y_rec")
        self.y_impact = tf.placeholder(tf.int32, [None], name="y_impact")
        self.y_substance = tf.placeholder(tf.int32, [None], name="y_substance")
        self.y_clarity = tf.placeholder(tf.int32, [None], name="y_clarity")
        self.y_confidence = tf.placeholder(tf.int32, [None], name="y_confidence")
        self.y_correctness = tf.placeholder(tf.int32, [None], name="y_correctness")
        self.y_originality = tf.placeholder(tf.int32, [None], name="y_originality")
        
        outputs = self.rnn_encoder_layer(X_embeddings, n_neurons, model_config.n_layers, model_config.cell_type, self.batch_seq_len, model_config.use_bidirectional)

        if model_config.use_extra_dense_layer:
          outputs = self.bn_drop_liner(outputs, n_neurons)
        
        rec_dense_layer = self.bn_drop_liner(outputs, n_neurons)
        impact_dense_layer = self.bn_drop_liner(outputs, n_neurons)
        substance_dense_layer = self.bn_drop_liner(outputs, n_neurons)
        clarity_dense_layer = self.bn_drop_liner(outputs, n_neurons)
        confidence_dense_layer = self.bn_drop_liner(outputs, n_neurons)
        correctness_dense_layer = self.bn_drop_liner(outputs, n_neurons)
        originality_dense_layer = self.bn_drop_liner(outputs, n_neurons)

        rec_dense_layer = self.bn_drop_liner(rec_dense_layer, n_neurons)
        impact_dense_layer = self.bn_drop_liner(impact_dense_layer, n_neurons)
        substance_dense_layer = self.bn_drop_liner(substance_dense_layer, n_neurons)
        clarity_dense_layer = self.bn_drop_liner(clarity_dense_layer, n_neurons)
        confidence_dense_layer = self.bn_drop_liner(confidence_dense_layer, n_neurons)
        correctness_dense_layer = self.bn_drop_liner(correctness_dense_layer, n_neurons)
        originality_dense_layer = self.bn_drop_liner(originality_dense_layer, n_neurons)
        
        rec_logits = self.bn_drop_liner_no_activation(rec_dense_layer, n_outputs)
        impact_logits = self.bn_drop_liner_no_activation(impact_dense_layer, n_outputs)
        substance_logits = self.bn_drop_liner_no_activation(substance_dense_layer, n_outputs)
        clarity_logits = self.bn_drop_liner_no_activation(clarity_dense_layer, n_outputs)
        confidence_logits = self.bn_drop_liner_no_activation(confidence_dense_layer, n_outputs)
        correctness_logits = self.bn_drop_liner_no_activation(correctness_dense_layer, n_outputs)
        originality_logits = self.bn_drop_liner_no_activation(originality_dense_layer, n_outputs)

        self.batch_rec_loss = self.get_loss(self.y_rec, rec_logits)
        self.batch_impact_loss = self.get_loss(self.y_impact, impact_logits)
        self.batch_substance_loss = self.get_loss(self.y_substance, substance_logits)
        self.batch_clarity_loss = self.get_loss(self.y_clarity, clarity_logits)
        self.batch_confidence_loss = self.get_loss(self.y_confidence, confidence_logits)
        self.batch_correctness_loss = self.get_loss(self.y_correctness, correctness_logits)
        self.batch_originality_loss = self.get_loss(self.y_originality, originality_logits)
        
        weighted_loss = tf.math.multiply([self.batch_rec_loss, self.batch_impact_loss, self.batch_substance_loss, self.batch_clarity_loss, self.batch_confidence_loss, self.batch_correctness_loss, self.batch_originality_loss], 
                                         model_config.aspect_weights)
        
        self.batch_loss = self.compute_list_mean(weighted_loss)
        
        self.optimize_loss(self.batch_loss, model_config.learning_rate, model_config.weight_decay)
        
        self.batch_rec_accuracy = self.get_accuracy(self.y_rec, rec_logits)
        self.batch_impact_accuracy = self.get_accuracy(self.y_impact, impact_logits)
        self.batch_substance_accuracy = self.get_accuracy(self.y_substance, substance_logits)
        self.batch_clarity_accuracy = self.get_accuracy(self.y_clarity, clarity_logits)
        self.batch_confidence_accuracy = self.get_accuracy(self.y_confidence, confidence_logits)
        self.batch_correctness_accuracy = self.get_accuracy(self.y_correctness, correctness_logits)
        self.batch_originality_accuracy = self.get_accuracy(self.y_originality, originality_logits)
        
        self.batch_accuracy = self.compute_list_mean([self.batch_impact_accuracy, self.batch_rec_accuracy, self.batch_substance_accuracy, self.batch_clarity_accuracy, self.batch_confidence_accuracy, self.batch_correctness_accuracy, self.batch_originality_accuracy])
        
        with tf.variable_scope(self.metrics_scope_name):
          self.rec_accuracy, rec_accuracy_update_op = self.get_epoch_accuracy(self.y_rec, rec_logits)
          self.impact_accuracy, impact_accuracy_update_op = self.get_epoch_accuracy(self.y_impact, impact_logits)
          self.substance_accuracy, substance_accuracy_update_op = self.get_epoch_accuracy(self.y_substance, substance_logits)
          self.clarity_accuracy, clarity_accuracy_update_op = self.get_epoch_accuracy(self.y_clarity, clarity_logits)
          self.confidence_accuracy, confidence_accuracy_update_op = self.get_epoch_accuracy(self.y_confidence, confidence_logits)
          self.correctness_accuracy, correctness_accuracy_update_op = self.get_epoch_accuracy(self.y_correctness, correctness_logits)
          self.originality_accuracy, originality_accuracy_update_op = self.get_epoch_accuracy(self.y_originality, originality_logits)
        
          self.rec_loss, rec_loss_update_op = tf.metrics.mean(self.batch_rec_loss)
          self.impact_loss, impact_loss_update_op = tf.metrics.mean(self.batch_impact_loss)
          self.substance_loss, substance_loss_update_op = tf.metrics.mean(self.batch_substance_loss)
          self.clarity_loss, clarity_loss_update_op = tf.metrics.mean(self.batch_clarity_loss)
          self.confidence_loss, confidence_loss_update_op = tf.metrics.mean(self.batch_confidence_loss)
          self.correctness_loss, correctness_loss_update_op = tf.metrics.mean(self.batch_correctness_loss)
          self.originality_loss, originality_loss_update_op = tf.metrics.mean(self.batch_originality_loss)
          
          self.rec_conf_matrix, rec_conf_matrix_update_op = self.get_epoch_conf_matrix(self.y_rec, rec_logits)
          self.impact_conf_matrix, impact_conf_matrix_update_op = self.get_epoch_conf_matrix(self.y_impact, impact_logits)
          self.substance_conf_matrix, substance_conf_matrix_update_op = self.get_epoch_conf_matrix(self.y_substance, substance_logits)
          self.clarity_conf_matrix, clarity_conf_matrix_update_op = self.get_epoch_conf_matrix(self.y_clarity, clarity_logits)
          self.confidence_conf_matrix, confidence_conf_matrix_update_op = self.get_epoch_conf_matrix(self.y_confidence, confidence_logits)
          self.correctness_conf_matrix, correctness_conf_matrix_update_op = self.get_epoch_conf_matrix(self.y_correctness, correctness_logits)
          self.originality_conf_matrix, originality_conf_matrix_update_op = self.get_epoch_conf_matrix(self.y_originality, originality_logits)
          
        self.update_metrics_op = tf.group([rec_accuracy_update_op, impact_accuracy_update_op, substance_accuracy_update_op, clarity_accuracy_update_op, 
                                          confidence_accuracy_update_op, correctness_accuracy_update_op, originality_accuracy_update_op,
                                          rec_loss_update_op, impact_loss_update_op, substance_loss_update_op, clarity_loss_update_op,
                                          confidence_loss_update_op, correctness_loss_update_op, originality_loss_update_op, 
                                          rec_conf_matrix_update_op, impact_conf_matrix_update_op, substance_conf_matrix_update_op, 
                                          clarity_conf_matrix_update_op, confidence_conf_matrix_update_op, correctness_conf_matrix_update_op,
                                          originality_conf_matrix_update_op])
        
        self.register_metrics_init_op()

        tf.summary.scalar("accuracy/rec", self.rec_accuracy)
        tf.summary.scalar("accuracy/impact", self.impact_accuracy)
        tf.summary.scalar("accuracy/subst", self.substance_accuracy)
        tf.summary.scalar("accuracy/clarity", self.clarity_accuracy)
        tf.summary.scalar("accuracy/confidence", self.confidence_accuracy)
        tf.summary.scalar("accuracy/correctness", self.correctness_accuracy)
        tf.summary.scalar("accuracy/originality", self.originality_accuracy)

        tf.summary.scalar("loss/rec", self.rec_loss)
        tf.summary.scalar("loss/impact", self.impact_loss)
        tf.summary.scalar("loss/substance", self.substance_loss)
        tf.summary.scalar("loss/clarity", self.clarity_loss)
        tf.summary.scalar("loss/confidence", self.confidence_loss)
        tf.summary.scalar("loss/correctness", self.correctness_loss)
        tf.summary.scalar("loss/originality", self.originality_loss)
        
        self.summary_op = tf.summary.merge_all()

        self.init = tf.global_variables_initializer()
        
  def bn_drop_liner(self, inputs, nr_units):
    outputs = self.batch_norm_layer(inputs)
    outputs = self.dropout_layer(outputs)
    outputs = fully_connected(outputs, nr_units)
    return outputs

  def bn_drop_liner_no_activation(self, inputs, nr_units):
    outputs = self.batch_norm_layer(inputs)
    outputs = self.dropout_layer(outputs)
    outputs = fully_connected(outputs, nr_units, activation_fn=None)
    return outputs
  
  def feed_dict_constructor(self, batch_comments, batch_label, batch_seq_len, droput_keep_prob, droput_keep_prob_input, droput_keep_prob_cell, is_training):
    return {self.X: batch_comments, 
            self.y_rec: batch_label["RECOMMENDATION"] - 1, 
            self.y_impact: batch_label["IMPACT"] - 1, 
            self.y_substance: batch_label["SUBSTANCE"] - 1,
            self.y_clarity: batch_label["CLARITY"] - 1,
            self.y_confidence: batch_label["REVIEWER_CONFIDENCE"] - 1,
            self.y_correctness: batch_label["SOUNDNESS_CORRECTNESS"] - 1,
            self.y_originality: batch_label["ORIGINALITY"] - 1,
            self.batch_seq_len: batch_seq_len,
            self.keep_prob:droput_keep_prob,
            self.keep_prob_input: droput_keep_prob_input,
            self.keep_prob_cell: droput_keep_prob_cell,
            self.is_training: is_training}
  
  def perform_train_op(self, sess, train_batch_item, droput_keep_prob, droput_keep_prob_input, droput_keep_prob_cell, aspect):
    sess.run(self.training_op, feed_dict=self.feed_dict_constructor(*train_batch_item, droput_keep_prob, droput_keep_prob_input, droput_keep_prob_cell, True))
    
  def update_batch_metrics(self, sess, batch_item, droput_keep_prob, aspect):
    sess.run(self.update_metrics_op, feed_dict=self.feed_dict_constructor(*batch_item, droput_keep_prob, droput_keep_prob, droput_keep_prob, False))
  
  def get_epoch_metrics(self, sess, aspect):
    aspect_ops_dict = {"RECOMMENDATION": [self.rec_conf_matrix, self.rec_accuracy, self.rec_loss],
                       "IMPACT": [self.impact_conf_matrix, self.impact_accuracy, self.impact_loss],
                       "SUBSTANCE": [self.substance_conf_matrix, self.substance_accuracy, self.substance_loss],
                       "CLARITY": [self.clarity_conf_matrix, self.clarity_accuracy, self.clarity_loss],
                       "REVIEWER_CONFIDENCE": [self.confidence_conf_matrix, self.confidence_accuracy, self.confidence_loss],
                         "SOUNDNESS_CORRECTNESS": [self.correctness_conf_matrix, self.correctness_accuracy, self.correctness_loss],
                       "ORIGINALITY": [self.originality_conf_matrix, self.originality_accuracy, self.originality_loss]}
          
    metric_op = aspect_ops_dict[str(aspect)] + [self.summary_op]

    conf_matrix, accuracy, train_loss, summary = sess.run(metric_op)
    return conf_matrix, accuracy, train_loss, summary

### RNN conditional model (cond review on aspect)

In [0]:
class RNN_conditional_model(RNN_base_model):
  def __init__(self, model_config, vocab_size, embedding_dim, n_steps):
    
    self.aspect_vocab_indx = {"RECOMMENDATION": vocab.stoi["suggestion"],
                              "IMPACT": vocab.stoi["impact"],
                               "SUBSTANCE": vocab.stoi["significant"],
                               "CLARITY": vocab.stoi["clarity"],
                               "REVIEWER_CONFIDENCE": vocab.stoi["confident"],
                               "SOUNDNESS_CORRECTNESS": vocab.stoi["correct"],
                               "ORIGINALITY": vocab.stoi["originality"]}
    
    with tf.device("/gpu:0"):
        super().__init__()
        
        embedding = self.embedding_layer_init(vocab_size, embedding_dim)
        
        self.aspect = tf.placeholder(tf.int32, [None, 1], name="aspect")
        
        aspect_embeddings = tf.nn.embedding_lookup(embedding, self.aspect)
        
        def cell_with_droput(layer_nr, name_suffix=""):
          cell_name = name_suffix + "cell_" + model_config.cell_type + "_" + str(layer_nr) 
          cell = self.get_cell(model_config.n_neurons, model_config.cell_type, cell_name)

          dropout_keep_prob = 1.0 if layer_nr == 0 else self.keep_prob

          return tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=dropout_keep_prob)        
        
        def get_named_cell(layer_nr, name_suffix=""):
          cell_name = name_suffix + "cell_" + model_config.cell_type + "_" + str(layer_nr) 
          return self.get_cell(model_config.n_neurons, model_config.cell_type, cell_name)

        cell_fw_aspect = tf.contrib.rnn.MultiRNNCell([get_named_cell(i, "fw_aspect") for i in range(1)])    
        cell_bw_aspect = tf.contrib.rnn.MultiRNNCell([get_named_cell(i, "bw_aspect") for i in range(1)])
        
        initial_state_fw = cell_fw_aspect.zero_state(tf.shape(self.aspect)[0], dtype=tf.float32)
        initial_state_bw = cell_bw_aspect.zero_state(tf.shape(self.aspect)[0], dtype=tf.float32)

        (init_fw_pass, init_bw_pass), _ = tf.nn.bidirectional_dynamic_rnn(cell_fw=cell_fw_aspect, cell_bw=cell_bw_aspect, inputs=aspect_embeddings, dtype=tf.float32, initial_state_fw=initial_state_fw, initial_state_bw=initial_state_bw)
        
        init_fw_pass = get_last_output(init_fw_pass)
        init_bw_pass = get_last_output(init_bw_pass)
        
        init_fw_pass = tuple([init_fw_pass for _ in range(model_config.n_layers)])
        init_bw_pass = tuple([init_bw_pass for _ in range(model_config.n_layers)])
        
        self.X = tf.placeholder(tf.int32, [None, n_steps], name="X")
        self.y = tf.placeholder(tf.int32, [None], name="y")

        X_embeddings = tf.nn.embedding_lookup(embedding, self.X)
        X_embeddings = self.input_dropout_layer(X_embeddings)
        
        cell_fw = tf.contrib.rnn.MultiRNNCell([cell_with_droput(i, "fw") for i in range(model_config.n_layers)])    
        cell_bw = tf.contrib.rnn.MultiRNNCell([cell_with_droput(i, "bw") for i in range(model_config.n_layers)])
        
        (fw_pass, bw_pass), states = tf.nn.bidirectional_dynamic_rnn(cell_fw=cell_fw, cell_bw=cell_bw, inputs=X_embeddings, dtype=tf.float32, sequence_length=self.batch_seq_len, initial_state_fw=init_fw_pass, initial_state_bw=init_bw_pass)

        outputs = tf.concat([get_last_output(fw_pass, self.batch_seq_len), get_last_output(bw_pass, self.batch_seq_len)], 1)
        
        if model_config.use_extra_dense_layer:
          outputs = self.dropout_layer(outputs)
          outputs = fully_connected(outputs, model_config.n_neurons)

        outputs = self.dropout_layer(outputs)
        outputs = fully_connected(outputs, model_config.n_neurons)  
          
        outputs = self.dropout_layer(outputs)
        logits = fully_connected(outputs, model_config.n_outputs, activation_fn=None)

        self.batch_loss = self.get_loss(self.y, logits)

        self.optimize_loss(self.batch_loss, model_config.learning_rate, model_config.weight_decay)

        self.batch_accuracy = self.get_accuracy(self.y, logits)
        
        with tf.variable_scope(self.metrics_scope_name):
          self.accuracy, accuracy_update_op = self.get_epoch_accuracy(self.y, logits)
          self.loss, loss_update_op = tf.metrics.mean(self.batch_loss)
          self.conf_matrix, conf_matrix_update_op = self.get_epoch_conf_matrix(self.y, logits)
  
        self.update_metrics_op = tf.group([accuracy_update_op, loss_update_op, conf_matrix_update_op])
        
        self.register_metrics_init_op()

        tf.summary.scalar("loss", self.loss)
        tf.summary.scalar("accuracy", self.accuracy)

        self.summary_op = tf.summary.merge_all()

        self.init = tf.global_variables_initializer()
        
  def feed_dict_constructor(self, batch_comments, batch_label, batch_seq_len, droput_keep_prob, droput_keep_prob_input, droput_keep_prob_cell, is_training, aspect):
    return {self.X: batch_comments, 
            self.y: batch_label[aspect] - 1,
            self.batch_seq_len: batch_seq_len,
            self.keep_prob:droput_keep_prob,
            self.keep_prob_input: droput_keep_prob_input,
            self.keep_prob_cell: droput_keep_prob_cell,
            self.is_training: is_training,
            self.aspect: [[self.aspect_vocab_indx[aspect]] for _ in range(len(batch_label[aspect]))]}
  
  def perform_train_op(self, sess, train_batch_item, droput_keep_prob, droput_keep_prob_input, droput_keep_prob_cell, sess_aspect):
    for aspect in env_var.aspects_no_com_approp:
      sess.run(self.training_op, feed_dict=self.feed_dict_constructor(*train_batch_item, droput_keep_prob, droput_keep_prob_input, droput_keep_prob_cell, True, aspect))
  
  
  def update_batch_metrics(self, sess, batch_item, droput_keep_prob, aspect):
    sess.run(self.update_metrics_op, feed_dict=self.feed_dict_constructor(*batch_item, droput_keep_prob, droput_keep_prob, droput_keep_prob, False, aspect))
    
  def get_epoch_metrics(self, sess, aspect):
    metric_op = [self.conf_matrix, self.accuracy, self.loss, self.summary_op]
    conf_matrix, accuracy, loss, summary = sess.run(metric_op)
    return conf_matrix, accuracy, loss, summary

### CNN

In [0]:
class CNN(RNN_base_model):
  def __init__(self, model_config, vocab_size, embedding_dim, n_steps):
    with tf.device('/gpu:0'):
      super().__init__()

      self.X = tf.placeholder(tf.int32, [None, n_steps], name="X")
      self.y = tf.placeholder(tf.int32, [None], name="y")

      embedding = self.embedding_layer_init(vocab_size, embedding_dim)       

      X_embeddings = tf.nn.embedding_lookup(embedding, self.X)

      X_embeddings = self.input_dropout_layer(X_embeddings)

      with tf.name_scope("cnn"):
        conv = tf.layers.conv1d(X_embeddings, model_config.num_filters, model_config.kernel_size)
        conv = tf.layers.max_pooling1d(conv,3,2)
        gmp = tf.reduce_max(conv, reduction_indices=[1])
        
      fc = tf.layers.dense(gmp, model_config.n_neurons)
      fc = tf.contrib.layers.dropout(fc, model_config.droput_keep_prob)
#       fc = self.batch_norm_layer(fc)
      fc = tf.nn.relu(fc)
      
      fc = tf.layers.dense(fc, model_config.n_neurons)
      fc = tf.contrib.layers.dropout(fc, model_config.droput_keep_prob)
#       fc = self.batch_norm_layer(fc)
      fc = tf.nn.relu(fc)
      
      logits = tf.layers.dense(fc, model_config.n_outputs)

      self.batch_loss = self.get_loss(self.y, logits)

      self.optimize_loss(self.batch_loss, model_config.learning_rate, model_config.weight_decay)

      self.batch_accuracy = self.get_accuracy(self.y, logits)

      with tf.variable_scope(self.metrics_scope_name):
        self.accuracy, accuracy_update_op = self.get_epoch_accuracy(self.y, logits)
        self.loss, loss_update_op = tf.metrics.mean(self.batch_loss)
        self.conf_matrix, conf_matrix_update_op = self.get_epoch_conf_matrix(self.y, logits)

      self.update_metrics_op = tf.group([accuracy_update_op, loss_update_op, conf_matrix_update_op])

      self.register_metrics_init_op()

      tf.summary.scalar("accuracy", self.accuracy)
      tf.summary.scalar("loss", self.loss)

      self.summary_op = tf.summary.merge_all()

      self.init = tf.global_variables_initializer()

  def feed_dict_constructor(self, batch_comments, batch_label, batch_seq_len, droput_keep_prob, droput_keep_prob_input, droput_keep_prob_cell, is_training, aspect):
    return {self.X: batch_comments, 
            self.y: batch_label[aspect] - 1,
            self.batch_seq_len: batch_seq_len,
            self.keep_prob:droput_keep_prob,
            self.keep_prob_input: droput_keep_prob_input,
            self.keep_prob_cell: droput_keep_prob_cell,
            self.is_training: is_training}

  def perform_train_op(self, sess, train_batch_item, droput_keep_prob, droput_keep_prob_input, droput_keep_prob_cell, aspect):
    sess.run(self.training_op, feed_dict=self.feed_dict_constructor(*train_batch_item, droput_keep_prob, droput_keep_prob_input, droput_keep_prob_cell, True, aspect))

  def update_batch_metrics(self, sess, batch_item, droput_keep_prob, aspect):
    sess.run(self.update_metrics_op, feed_dict=self.feed_dict_constructor(*batch_item, droput_keep_prob, droput_keep_prob, droput_keep_prob, False, aspect))

  def get_epoch_metrics(self, sess, aspect):
    metric_op = [self.conf_matrix, self.accuracy, self.loss, self.summary_op]
    conf_matrix, accuracy, loss, summary = sess.run(metric_op)
    return conf_matrix, accuracy, loss, summary

## Trainer

#### Helper funs

In [0]:
# https://github.com/ageron/handson-ml/blob/master/11_deep_learning.ipynb

def _get_model_params(sess):
    """Get all variable values (used for early stopping, faster than saving to disk)"""
    gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    return {gvar.op.name: value for gvar, value in zip(gvars, sess.run(gvars))}

def _restore_model_params(sess, model_params):
    """Set all variables to the given values (for early stopping, faster than loading from disk)"""
    gvar_names = list(model_params.keys())
    assign_ops = {gvar_name: tf.get_default_graph().get_operation_by_name(gvar_name + "/Assign") for gvar_name in gvar_names}
    init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
    feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
    sess.run(assign_ops, feed_dict=feed_dict)

In [0]:
class Epoch_Metrics():
  def __init__(self, conf_matrix, accuracy, loss):
    self.conf_matrix = conf_matrix
    self.accuracy = accuracy
    self.loss = loss
    
    
class Stat():
  def __init__(self, epoch, train_metrics, dev_metrics, test_metrics, iteration):
    self.epoch = epoch    
    self.train_accuracy = train_metrics.accuracy
    self.dev_accuracy = dev_metrics.accuracy
    self.train_loss = train_metrics.loss
    self.dev_loss = dev_metrics.loss
    self.test_accuracy = test_metrics.accuracy
    self.dev_conf_matrix = dev_metrics.conf_matrix
    self.test_conf_matrix = test_metrics.conf_matrix
    self.iteration = iteration
    
  def __str__(self):
    return f'{self.epoch} Train accuracy: {self.train_accuracy} Dev accuracy: {self.dev_accuracy} Train loss: {self.train_loss} Dev loss: {self.dev_loss}, Test acc: {self.test_accuracy}'


In [0]:
class Train_Stat_Tracker():
  def __init__(self):
    
    self.best_dev_accuracy = 0
    self.best_dev_acc_epoch = 0
    self.best_dev_loss_epoch = 0
    self.best_dev_loss = np.infty
    
    self.best_params = None

    self.checks_without_progress = 0
    self.max_checks_without_progress = 10
    
    self.nr_epoch_neigh = 2
    
    self.train_stats = []
    self.best_epoch_stats = None
      
  def record_train_stats(self, sess, epoch, train_metrics, dev_metrics, test_metrics, iteration):

    epoch_stat = Stat(epoch, train_metrics, dev_metrics, test_metrics, iteration)

    self.train_stats.append(epoch_stat)

    if epoch_stat.train_loss < epoch_stat.dev_loss and epoch_stat.train_accuracy >= epoch_stat.dev_accuracy:

      if self.best_dev_accuracy < epoch_stat.dev_accuracy:
        self.best_dev_accuracy = epoch_stat.dev_accuracy
        self.best_dev_acc_epoch = epoch

      if self.best_dev_loss > epoch_stat.dev_loss:
        self.best_dev_loss = epoch_stat.dev_loss
        self.best_dev_loss_epoch = epoch
#         self.best_params = _get_model_params(sess)   
        self.checks_without_progress = 0
      else:
        self.checks_without_progress += 1
    
    return epoch_stat
  
  def check_early_stopping(self, is_last_epoch):
    should_early_stop = False

    if self.checks_without_progress > self.max_checks_without_progress or is_last_epoch:
      
      left_window = self.best_dev_loss_epoch - self.nr_epoch_neigh

      if left_window < 0: 
        left_window = 0

      right_window = self.best_dev_loss_epoch + self.nr_epoch_neigh + 1
      
      best_loss_neigh_epoch_stats = self.train_stats[left_window:right_window]
      
      best_epoch_stats = self.train_stats[self.best_dev_acc_epoch]
      
      if len(best_loss_neigh_epoch_stats) > 1:
        best_loss_neigh_best_acc = max(best_loss_neigh_epoch_stats, key=attrgetter("dev_accuracy"))

        best_epoch_stats = self.train_stats[self.best_dev_loss_epoch]

        if best_loss_neigh_best_acc.dev_accuracy > best_epoch_stats.dev_accuracy and best_loss_neigh_best_acc.train_accuracy >= best_loss_neigh_best_acc.dev_accuracy and best_loss_neigh_best_acc.train_loss <= best_loss_neigh_best_acc.dev_loss:
          best_epoch_stats = best_loss_neigh_best_acc
      
      self.best_epoch_stats = best_epoch_stats

      should_early_stop = True
    
    return should_early_stop

#### Trainer

In [0]:
from operator import itemgetter, attrgetter

def get_epoch_metrics(sess, model, batch_it, aspect_label_name):
  sess.run(model.metrics_init_op)
  
  for batch_item in batch_it:
    model.update_batch_metrics(sess, batch_item, 1., aspect_label_name)
  
  conf_matrix, accuracy, loss, summary = model.get_epoch_metrics(sess, aspect_label_name)
  
  return Epoch_Metrics(conf_matrix, accuracy, loss), summary
  
def trainer(model, model_config, trainer_config, train_stat_tracker, vocab, aspect_label_name, logdir_aspect_name, iteration, verbose=False):
    
  train_file_writer = tf.summary.FileWriter(logdir_aspect_name / "train", tf.get_default_graph())            
  dev_file_writer = tf.summary.FileWriter(logdir_aspect_name / "dev", tf.get_default_graph())
  saver = tf.train.Saver()
  model_checkpoint_exists = tf.train.checkpoint_exists(trainer_config.saver_path)
    
  with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:

      if model_checkpoint_exists:
        saver.restore(sess, trainer_config.saver_path)
      else:
        model.init.run()

        train_set_vocab = vocab.vectors.numpy()
        sess.run(model.embedding_init, feed_dict={model.embedding_placeholder: train_set_vocab})
                                                                                              
      for epoch in range(trainer_config.n_epochs):
          
          for train_batch_item in train_batch_it:
            model.perform_train_op(sess, train_batch_item, model_config.droput_keep_prob, model_config.droput_keep_prob_input, model_config.droput_keep_prob_cell, aspect_label_name)
          
          train_metrics, train_summary = get_epoch_metrics(sess, model, train_batch_it, aspect_label_name)
          dev_metrics, dev_summary = get_epoch_metrics(sess, model, dev_batch_it, aspect_label_name)
          test_metrics, test_summary = get_epoch_metrics(sess, model, test_batch_it, aspect_label_name)
          
          train_file_writer.add_summary(train_summary, epoch)
          dev_file_writer.add_summary(dev_summary, epoch)

#           train_file_writer.flush()
#           dev_file_writer.flush()

          if epoch != 0 and epoch % trainer_config.nr_epochs_to_save_model_checkpoint == 0:
            saver.save(sess, trainer_config.saver_path)

            print("Checkpoint saved!")
        
          epoch_stat = train_stat_tracker.record_train_stats(sess, epoch, train_metrics, dev_metrics, test_metrics, iteration)
          
          if verbose:
            print(epoch_stat)
          
          is_last_epoch = epoch + 1 == trainer_config.n_epochs
          
          if train_stat_tracker.check_early_stopping(is_last_epoch):
            
            print("best_epoch_stats")
            print(train_stat_tracker.best_epoch_stats)
            break
      
      if train_stat_tracker.best_params:
        print("Restoring model weights...")
        _restore_model_params(sess, train_stat_tracker.best_params)
        train_metrics, train_summary = get_epoch_metrics(sess, model, train_batch_it, aspect_label_name)
        dev_metrics, dev_summary = get_epoch_metrics(sess, model, dev_batch_it, aspect_label_name)
        test_metrics, test_summary = get_epoch_metrics(sess, model, test_batch_it, aspect_label_name)
#         print("Train accuracy:", train_accuracy, "Dev accuracy", dev_accuracy, "Train loss:", train_loss, "Dev loss:", dev_loss, "Test acc:", test_accuracy)
      
      train_file_writer.close()
      dev_file_writer.close()

## Main

#### Config

In [0]:
# Single aspect learning
class RNN_config(object):
  n_neurons = 100
  n_outputs = 5
  learning_rate = 0.005
  n_layers = 2
  cell_type = "gru" 
  use_bidirectional = True
  use_extra_dense_layer = True
  weight_decay = 0.01
  
  droput_keep_prob=0.9
  droput_keep_prob_input = 1.0
  droput_keep_prob_cell = 0.5
  
# MTL
class MTL_RNN_config(object):
  n_neurons = 100
  n_outputs = 5
  learning_rate = 0.005
  n_layers = 2
  cell_type = "gru" 
  use_bidirectional = True
  use_extra_dense_layer = True
  weight_decay = 0

  droput_keep_prob = 0.9
  droput_keep_prob_input = 1.0
  droput_keep_prob_cell = 0.5
  
  aspect_weights = [1.0] * 7
  
# Conditional
class Cond_RNN_config(object):
  n_neurons = 100
  n_outputs = 5
  learning_rate = 0.005
  n_layers = 2
  cell_type = "gru"
  use_extra_dense_layer = True
  weight_decay = 0

  droput_keep_prob = 0.5
  droput_keep_prob_input = 1.0
  droput_keep_prob_cell = 0.5
  
# CNN
class CNNConfig(object):
  n_neurons = 100
  n_outputs = 5
  
  num_filters = 128
  kernel_size = 5
  
  learning_rate = 1e-3

  weight_decay = 0

  droput_keep_prob=0.5
  droput_keep_prob_input = 0.9
  droput_keep_prob_cell = 0.5
  n_layers = 2
  cell_type = "gru" 
  use_bidirectional = True

class Trainer_config(object):
  def __init__(self):
    self.n_epochs = 50
    self.nr_epochs_to_save_model_checkpoint = 300
    self.saver_path = "model_ckpt_y_rec_impact_clarity"
    self.logdir = Path("multiple_aspects")
    self.runs = 10
    self.stats_runs_path = env_var.stats_path / "tensorflow_models_stats.p"
#     self.stats_runs_path = Path("tensorflow_models_stats.p")
    
  def set_logdir(self, path):
    self.logdir = env_var.stats_path / "tboard" / path

class Preproc_config(object):
  seq_length = 350 #n_steps
  vocab_max_size = 20000
  vocab_min_freq = 5
  pretrained_vectors = torchtext.vocab.Vectors("glove.6B.100d.txt", env_var.embeddings_path)
  comments_field_name = 'comments'

In [0]:
def get_model(model_name=None):
  if model_name == "MTL_RNN":
    return RNN_multi_task_model, MTL_RNN_config
  
  if model_name == "Cond_RNN":
    return RNN_conditional_model, Cond_RNN_config
  
  if model_name == "CNN":
    return CNN, CNNConfig
  
  return RNN_single_aspect_model, RNN_config

#### Tensorboard

In [0]:
# import shutil
# shutil.rmtree("gdrive/My Drive/thesis/tensorflow_stats/tboard")

In [0]:
# %tensorboard --logdir {Trainer_config.logdir}
%tensorboard --logdir 'gdrive/My Drive/thesis/tensorflow_stats_random/tboard/CNN/recommendation'

#### Main run

In [0]:
dataset_intermediate_paths =[Path(""),
                             Path("strength_weak_sections"),
                             Path("manual_labeled_strength_weak_sections"),
                             Path("manual_labeled_strength_weak_sections/first_section"),
                             Path("manual_labeled_strength_weak_sections/strength_weak_sections"),
                             Path("manual_labeled_strength_weak_sections/strength_weak_sections_len_limit"),
                             Path("manual_labeled_strength_weak_sections/summary_strength_weak_sections"),
                             Path("manual_labeled_strength_weak_sections/summary_strength_weak_sections/clarity_sentences"),
                             Path("manual_labeled_strength_weak_sections/weak_section"),
                             Path("manual_labeled_strength_weak_sections/weak_strength_sections_len_limit"),
                             Path("manual_labeled_strength_weak_sections/aug_stren_weak_sections_len_limit"),
                             Path("manual_labeled_strength_weak_sections/strength_section"),
                             Path("acl_abstracts")
                            ]

In [0]:
import operator

class Model_stats(object):
  def __init__(self, dataset_intermediate_path, model_name, aspect_label_name):
    self.dataset_intermediate_path = dataset_intermediate_path
    self.model_name = model_name
    self.aspect_label_name = aspect_label_name
    self.stats = []
    
  def compute_mean_metrics(self):
    top_3_dev_acc_stats = sorted(self.stats, key=operator.attrgetter('dev_accuracy'), reverse=True)[:3]
    
    self.mean_convergence_epoch = np.mean([stat.epoch for stat in self.stats]) 
    
    self.mean_dev_accuracy = np.mean([stat.dev_accuracy for stat in self.stats]) 
    self.mean_test_accuracy = np.mean([stat.test_accuracy for stat in self.stats]) 
    
    self.std_dev_accuracy = np.std([stat.dev_accuracy for stat in self.stats]) 
    self.std_test_accuracy = np.std([stat.test_accuracy for stat in self.stats]) 
    
    self.top_3_mean_dev_accuracy = np.mean([stat.dev_accuracy for stat in top_3_dev_acc_stats]) 
    self.top_3_mean_test_accuracy = np.mean([stat.test_accuracy for stat in top_3_dev_acc_stats]) 
    
    self.best_dev_acc_stats = top_3_dev_acc_stats[0]
    self.best_epoch_dev_conf_matrix = self.best_dev_acc_stats.dev_conf_matrix
    self.best_epoch_test_conf_matrix = self.best_dev_acc_stats.test_conf_matrix    

In [0]:
import pickle

class Stats_Manager(object):
  def __init__(self, stats_runs_path):
    self.stats_runs_path = stats_runs_path
  
  def create_or_restore_runs_stats_obj(self, dataset_intermediate_paths, model_names, aspect_label_names):
    self.models_stats = []

    if self.stats_runs_path.is_file():
      with open(self.stats_runs_path, "rb") as input_file:
        self.models_stats = pickle.load(input_file)

    self.should_restore_to_last_run = len(self.models_stats)

    if self.should_restore_to_last_run:
      last_run = self.models_stats[-1]

      self.last_path_idx = dataset_intermediate_paths.index(last_run.dataset_intermediate_path)
      self.last_model_idx = model_names.index(last_run.model_name)
      self.last_aspect_idx = aspect_label_names.index(last_run.aspect_label_name)

  def should_skip_current_run(self):
    _should_skip_current_run = False
    
    if self.should_restore_to_last_run:
      _should_skip_current_run = path_idx < self.last_path_idx or (path_idx <= self.last_path_idx and model_idx < self.last_model_idx) or (path_idx <= self.last_path_idx and model_idx <= self.last_model_idx and aspect_idx <= self.last_aspect_idx)
    
    return _should_skip_current_run
  
  def update(self, model_stats):
    self.models_stats.append(model_stats)
    with open(self.stats_runs_path, "wb") as output_file:
      pickle.dump(self.models_stats, output_file)    

In [0]:
model_names = ["CNN", "RNN", "MTL_RNN", "Cond_RNN"]
aspect_label_names = ['RECOMMENDATION', 'IMPACT', 'SUBSTANCE', 'CLARITY', 'REVIEWER_CONFIDENCE', 'SOUNDNESS_CORRECTNESS', 'ORIGINALITY']

preproc_config = Preproc_config()
trainer_config = Trainer_config()

stats_manager = Stats_Manager(trainer_config.stats_runs_path)
stats_manager.create_or_restore_runs_stats_obj(dataset_intermediate_paths, model_names, aspect_label_names)
  
for path_idx, ds_inter_path in enumerate(dataset_intermediate_paths):
  
  env_var.set_dataset_intermediate_path(ds_inter_path)
  trainer_config.set_logdir(ds_inter_path)
  
  print("DS_PATH:", env_var.dataset_path)
  
  for model_idx, model_name in enumerate(model_names):
    print(model_name)
    for aspect_idx, aspect_label_name in enumerate(aspect_label_names):
      print(aspect_label_name)
      
      if stats_manager.should_skip_current_run():
        continue
      
      model_stats = Model_stats(ds_inter_path, model_name, aspect_label_name)
      
      for iteration in range(trainer_config.runs):
        env_var.set_ds_fname(iteration)
        
        print(env_var.train_dataset_filename, env_var.dev_dataset_filename)
        
        vocab, vocab_size, embedding_dim, train_batch_it, dev_batch_it, test_batch_it = preprocessing_pipeline(preproc_config.seq_length, preproc_config.vocab_max_size, preproc_config.vocab_min_freq, preproc_config.pretrained_vectors, preproc_config.comments_field_name)
                
        tf.reset_default_graph()
    
        model, model_config = get_model(model_name)

        model = model(model_config, vocab_size, embedding_dim, preproc_config.seq_length)

        logdir_aspect_name = trainer_config.logdir/ model_name/ aspect_label_name.lower() / str(iteration)
      
        train_stat_tracker = Train_Stat_Tracker()

        trainer(model, model_config, trainer_config, train_stat_tracker, vocab, aspect_label_name, logdir_aspect_name, iteration, verbose=False)
        
        model_stats.stats.append(train_stat_tracker.best_epoch_stats)
        
      model_stats.compute_mean_metrics()
      stats_manager.update(model_stats)

#### Model summary

In [24]:
preproc_config = Preproc_config()
trainer_config = Trainer_config()

ds_inter_path = dataset_intermediate_paths[0]
env_var.set_dataset_intermediate_path(ds_inter_path)
trainer_config.set_logdir(ds_inter_path)

print("DS_PATH:", env_var.dataset_path)

vocab, vocab_size, embedding_dim, train_batch_it, dev_batch_it, test_batch_it = preprocessing_pipeline(preproc_config.seq_length, preproc_config.vocab_max_size, preproc_config.vocab_min_freq, preproc_config.pretrained_vectors, preproc_config.comments_field_name)
  
model_names = ["CNN", "RNN", "MTL_RNN", "Cond_RNN"]


DS_PATH: gdrive/My Drive/thesis/data


In [32]:
import tensorflow.contrib.slim as slim

tf.reset_default_graph()

model_name = "Cond_RNN"

model, model_config = get_model(model_name)
model = model(model_config, vocab_size, embedding_dim, preproc_config.seq_length)

def model_summary():
  model_vars = tf.trainable_variables()
  slim.model_analyzer.analyze_vars(model_vars, print_info=True)
#   slim.model_analyzer.analyze_ops(tf.get_default_graph(), print_info=True)
# dir(slim)
model_summary()

---------
Variables: name (type shape) [size]
---------
embedding:0 (float32_ref 1926x100) [192600, bytes: 770400]
bidirectional_rnn/fw/multi_rnn_cell/cell_0/fw_aspectcell_gru_0/gates/kernel:0 (float32_ref 200x200) [40000, bytes: 160000]
bidirectional_rnn/fw/multi_rnn_cell/cell_0/fw_aspectcell_gru_0/gates/bias:0 (float32_ref 200) [200, bytes: 800]
bidirectional_rnn/fw/multi_rnn_cell/cell_0/fw_aspectcell_gru_0/candidate/kernel:0 (float32_ref 200x100) [20000, bytes: 80000]
bidirectional_rnn/fw/multi_rnn_cell/cell_0/fw_aspectcell_gru_0/candidate/bias:0 (float32_ref 100) [100, bytes: 400]
bidirectional_rnn/bw/multi_rnn_cell/cell_0/bw_aspectcell_gru_0/gates/kernel:0 (float32_ref 200x200) [40000, bytes: 160000]
bidirectional_rnn/bw/multi_rnn_cell/cell_0/bw_aspectcell_gru_0/gates/bias:0 (float32_ref 200) [200, bytes: 800]
bidirectional_rnn/bw/multi_rnn_cell/cell_0/bw_aspectcell_gru_0/candidate/kernel:0 (float32_ref 200x100) [20000, bytes: 80000]
bidirectional_rnn/bw/multi_rnn_cell/cell_0/bw_a